In [1]:
import rpy2.robjects as ro
import rpy2.robjects.packages as rpackages
import anndata as ad
import scanpy as sc
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import pandas as pd
import umap
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from keras.callbacks import EarlyStopping
import seaborn as sb
from scipy.spatial.distance import pdist
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
import time
from keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.decomposition import PCA
import math
import random
import rpy2.robjects as ro
from rpy2 import robjects
from rpy2.robjects import r
from rpy2.robjects import pandas2ri
import pandas as pd
import pydeseq2

# Enable automatic conversion between R objects and pandas DataFrames
pandas2ri.activate()

2025-01-06 11:21:29.204925: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#import the data

# Function to read RDS file and extract counts and metadata
def read_rds_to_matrix_and_metadata(file_path):
    # Load the RDS file in R
    ro.r(f"sce <- readRDS('{file_path}')")

    # Extract count data (assumed to be stored in assays)
    counts = ro.r('assay(sce, "scalelogcounts")')
    # Extract row (gene) and column (cell) names
    gene_names = ro.r('rownames(sce)')
    cell_names = ro.r('colnames(sce)')
    
    # Convert to a NumPy array
    counts_np = ro.conversion.rpy2py(counts)

    # Convert the counts matrix to a pandas DataFrame
    counts_df = pd.DataFrame(counts_np, index=gene_names, columns=cell_names)

    # Extract metadata from colData and convert to a pandas DataFrame directly
    metadata = ro.r('as.data.frame(colData(sce))')  # Get the colData as an R data frame
    metadata_df = pd.DataFrame(metadata)  # Convert R data frame to pandas DataFrame directly

    return counts_df, metadata_df

# Usage example
file_path = '/tmp/work/RCproject/GEO_singlecellexperiment.rds'
counts_df, metadata_df = read_rds_to_matrix_and_metadata(file_path)

# Display the results
print("Counts DataFrame:")
print(counts_df.head())  # Show the first few rows of the counts DataFrame
print("\nMetadata DataFrame:")
print(metadata_df.head())  # Show the first few rows of the metadata DataFrame

R[write to console]: Loading required package: SingleCellExperiment

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowC

Counts DataFrame:
        GSM3899156_GSE133057  GSM3899157_GSE133057  GSM3899158_GSE133057  \
A1CF                0.057669             -0.027535              0.447468   
A2M                 0.392288             -0.233300             -0.691535   
A2ML1              -0.448972             -0.560902             -0.670516   
A4GALT             -0.218099              0.224682             -0.247762   
AAAS               -1.148204             -0.252932              0.053839   

        GSM3899159_GSE133057  GSM3899160_GSE133057  GSM3899161_GSE133057  \
A1CF                0.045675              0.014587              1.406735   
A2M                 0.293834             -1.091569             -0.816868   
A2ML1               0.532968              0.516362              1.266745   
A4GALT             -1.663294             -0.673025              0.513021   
AAAS                0.442668              0.006890              0.582186   

        GSM3899162_GSE133057  GSM3899163_GSE133057  GSM3899164_GSE13

In [28]:
filtered_metadata_df = metadata_df[~metadata_df['Response'].isin(['partial'])].copy()
row_names = filtered_metadata_df.index.tolist()  # Convert index to a list
filtered_counts_df = counts_df[row_names].copy()

# # Replace NaN values with 0
# filtered_counts_df = filtered_counts_df.fillna(0)

# Ensure all values are integers
# filtered_counts_df = filtered_counts_df.astype(int)

In [33]:
from scipy.stats import ttest_ind

# Example: Two conditions
condition1 = filtered_counts_df.loc[:, filtered_metadata_df['Response'] == 'yes']
condition2 = filtered_counts_df.loc[:, filtered_metadata_df['Response'] == 'no']

# Apply t-test for each gene
p_values = []
for gene in filtered_counts_df.index:
    t_stat, p_val = ttest_ind(condition1.loc[gene], condition2.loc[gene], equal_var=False)
    p_values.append(p_val)

# Adjust for multiple testing using FDR
from statsmodels.stats.multitest import multipletests
adjusted_pvals = multipletests(p_values, method='fdr_bh')[1]

In [43]:
# Example: Genes and their p-values
genes = filtered_counts_df.index  # List of gene names
p_values_df = pd.DataFrame({'Gene': genes, 'P_value': adjusted_pvals})

# Filter genes with p-value < 0.1
filtered_genes = p_values_df[p_values_df['P_value'] < 0.01]

# Get a list of the gene names
filtered_gene_list = filtered_genes['Gene'].tolist()

# Display the filtered genes
print(f"Number of genes with p-value < 0.01: {len(filtered_gene_list)}")
print(filtered_gene_list)

Number of genes with p-value < 0.01: 421
['ACADVL', 'ACSF3', 'ACTB', 'ACTN4', 'ADAM15', 'ADCY2', 'ADCY6', 'ADGRB1', 'AGO2', 'AGRN', 'ANAPC2', 'ANKLE2', 'ANO9', 'AP1G1', 'AP3D1', 'ARFGAP1', 'ARFRP1', 'ARHGAP27', 'ARHGAP32', 'ARHGEF1', 'ARHGEF11', 'ARHGEF2', 'ARL6IP6', 'ARPC2', 'ARSK', 'ASPH', 'ATF6', 'ATG13', 'ATG16L2', 'ATG4C', 'ATP13A2', 'ATP23', 'ATP2A2', 'ATP6V0A1', 'BACE2', 'BAG6', 'BAIAP2L1', 'BAZ2A', 'BCAS2', 'BLCAP', 'BLOC1S4', 'BRD2', 'BRD4', 'BRD9', 'BSG', 'C12orf60', 'C18orf54', 'C4orf46', 'CALR', 'CBY1', 'CCDC121', 'CCDC14', 'CD44', 'CD47', 'CD55', 'CDC37', 'CDC42BPB', 'CDCP1', 'CDH17', 'CDK10', 'CDK16', 'CDKN1A', 'CDNF', 'CEACAM1', 'CEACAM5', 'CEACAM6', 'CEMIP', 'CGN', 'CHPF', 'CIC', 'CLDN4', 'CLDN7', 'CLIC1', 'CLIP2', 'CLOCK', 'CLP1', 'CLUH', 'CNOT1', 'COMMD1', 'COMMD8', 'CREB3L2', 'CSNK1D', 'CSNK1G2', 'CTF1', 'CTNNA1', 'CTSZ', 'CUX1', 'CXCL11', 'DAB2IP', 'DBNL', 'DCAF8', 'DCK', 'DCP1B', 'DCTN5', 'DENND3', 'DHCR24', 'DHX30', 'DIAPH1', 'DIDO1', 'DLGAP4', 'DMWD', 'DNAH1', 'D